In [ ]:
import os
import re
import csv
import math
import time
import pandas as pd
from random import randint
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.firefox.options import Options

def write_to_file(outfile, var, header):
    if os.path.isfile(outfile):
        with open(outfile, 'a', newline='',  encoding='"utf-8-sig"') as result:
            writer = csv.writer(result)
            writer.writerow(var)
    else:
        with open(outfile, 'w', newline='', encoding='"utf-8-sig"') as result:
            writer = csv.writer(result)
            writer.writerow(header)
            writer.writerow(var)

# Oliver's Market

In [ ]:
def olivers_market_scraper(outfile):
    options = Options()
    options.headless = True

    driver = webdriver.Firefox(options=options)
    driver.get('https://www.oliversmarket.com/recipe/')

    next_page = driver.find_element_by_xpath("//div[@class='paginate clearfix']//descendant::a").get_attribute('href')[:-1]
    last_page = int(driver.find_element_by_xpath("//span[@class='page-numbers dots']//following-sibling::a").text)

    all_recipe_links = {}
    page = 1
    while page <= last_page:
        page += 1

        recipe_tags = driver.find_elements_by_xpath("//div[@class='card-info']//descendant::a")
        recipe_links = {recipe_tag.text: recipe_tag.get_attribute('href') for recipe_tag in recipe_tags}
        all_recipe_links.update(recipe_links)

        time.sleep(randint(5,8))

        driver.get(f'{next_page}{page}')
    
    recipe_names = list(all_recipe_links.keys())
    recipe_links = list(all_recipe_links.values())

    index = 100000
    for counter in range(0, len(all_recipe_links)):
        driver.get(recipe_links[counter])
        try:
            recipe_description = driver.find_element_by_xpath("//div[@class='single-intro']//descendant::p").text
        except NoSuchElementException:
            recipe_description = None
        try:
            recipe_image = driver.find_element_by_xpath("//div[@class='post-featured-image']//descendant::img").get_attribute('src')
        except NoSuchElementException:
            recipe_image = None
        try:
            recipe_overview = driver.find_element_by_xpath("//ul[@class='recipe-details']").text.splitlines()
        except NoSuchElementException:
            recipe_overview = None
        try:
            recipe_ingredients = driver.find_element_by_xpath("//ul[@class='ingredients clearfix']").text.splitlines()
        except NoSuchElementException:
            recipe_ingredients = None
        try:
            recipe_instructions = driver.find_element_by_xpath("//ol[@class='instructions clearfix']").text.splitlines()
        except NoSuchElementException:
            recipe_instructions = None

        header = ['olivers_market_id', 'recipe_name', 'recipe_link', 'recipe_description', 'recipe_image', 'recipe_overview', 'recipe_ingredients', 'recipe_instructions']
        var = [index, recipe_names[counter], recipe_links[counter], recipe_description, recipe_image, recipe_overview, recipe_ingredients, recipe_instructions]
        write_to_file(outfile, var=var, header=header)

        index += 1
        
        time.sleep(randint(5,8))

    driver.close()
    
outfile = '' # Insert desired outfile
olivers_market_scraper(outfile)

# Publix

In [ ]:
import re
category_regex = re.compile(r'(.*) \(\d*\)')

driver = webdriver.Firefox()
driver.get('https://ww4.publix.com/recipes-planning/browse-all-recipes?results=75')
category_pages = driver.find_elements_by_xpath("//ul[@class='category-list']/descendant::li/descendant::a")
category_n_link = {category_regex.search(category_page.text).group(1):category_page.get_attribute('href') for category_page in category_pages}

category = list(category_n_link.keys())
category_link = list(category_n_link.values())

# for counter in range(0, len(category_n_link)):
#     driver.get(category_link[counter])

In [ ]:
driver.get(category_link[0])

recipe_pages = driver.find_elements_by_xpath("//div[@class='pod pod-links recipe-card card-fourth']//descendant::div/descendant::h4/descendant::a")
recipe_n_link = {recipe_page.text:recipe_page.get_attribute('href') for recipe_page in recipe_pages}

recipe_images = driver.find_elements_by_xpath("//div[@class='pod pod-links recipe-card card-fourth']//descendant::a//descendant::img")
recipe_images = [recipe_image.get_attribute('src') if 'no-image' not in recipe_image.get_attribute('src') else None for recipe_image in recipe_images]

recipe_names = list(recipe_n_link.keys())
recipe_links = list(recipe_n_link.values())

# for counter in range(0, len(recipe_n_link)):
#     driver.get(recipe_links[counter])

In [ ]:
driver = webdriver.Firefox()
driver.get(recipe_links[0])

try:
    recipe_total_time = driver.find_element_by_xpath("//dd[@id='content_1_ddTotalTime']").text
except NoSuchElementException:
    recipe_total_time = None
try:
    recipe_serving = driver.find_element_by_xpath("//dd[@id='content_1_ddServings']").text
except NoSuchElementException:
    recipe_serving = None
try:
    recipe_active_time = driver.find_element_by_xpath("//dd[@id='content_1_ddActiveTime']").text
except NoSuchElementException:
    recipe_active_time = None
try:
    recipe = driver.find_elements_by_xpath("//div[@class=recipe-tab-pane ui-tabs-panel ui-widget-content ui-corner-bottom']")
except NoSuchElementException:
    recipe = None
# try:
#     recipe_nutrition = driver.find_element_by_xpath("//div[@id='nutrition']").text
# except NoSuchElementException:
#     recipe_nutrition = None

# Tesco

In [ ]:
# def olivers_market_scraper(outfile):

url = 'https://realfood.tesco.com/search.html?&perpage=90'
t_n_t_outfile = 'testco_tips_n_trends.csv'
tesco_results = 'tesco_recipes.csv'


options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)
driver.get(url)

last_page = driver.find_element_by_xpath("//a[@class='ddl-search-pagination__link ddl-search-pagination__btn ddl-search-pagination__btn_last']").get_attribute('href')
driver.get(last_page)
last_page_num = int(driver.find_element_by_xpath("//a[@class='ddl-search-pagination__link ddl-search-pagination__link_active']").text)
driver.get(url)

page_num = 1
recipe_links = []
recipe_types = []
recipe_names = []
while page_num <= last_page_num:
    
    page_recipe_links_eles = driver.find_elements_by_xpath("//a[@class='ddl-search-results__item-link']")
    page_recipe_links = [recipe_link.get_attribute('href') for recipe_link in page_recipe_links_eles]
    recipe_links = recipe_links + page_recipe_links
    
    page_recipe_types_eles = driver.find_elements_by_xpath("//div[@class='ddl-search-results__item-category']")
    page_recipe_types = [recipe_type.text for recipe_type in page_recipe_types_eles]
    recipe_types = recipe_types + page_recipe_types

    page_recipe_names_eles = driver.find_elements_by_xpath("//h2[@class='ddl-search-results__item-heading']")
    page_recipe_names = [recipe_name.text for recipe_name in page_recipe_names_eles]
    recipe_names = recipe_names + page_recipe_names
    
    page_num += 1
    
    next_page = f'https://realfood.tesco.com/search.html?&page={page_num}&perpage=90'
    driver.get(next_page)
    
    

index = 100000
for counter in range(0, len(recipe_links)):
    if recipe_types[counter] != 'Recipe':
        var = [recipe_names[counter], recipe_links[counter], recipe_types[counter]]
        header = ['recipe_name', 'recipe_link', 'recipe_type']
        
        write_to_file(t_n_t_outfile, var, header)
    else:
        driver.get(recipe_links[counter])

        try:
            recipe_description = driver.find_element_by_xpath("//p[@class='recipe-detail__intro']").text.replace(' See method', '')
        except NoSuchElementException:
            recipe_description = None
        try:
            recipe_ingredients = driver.find_element_by_xpath("//ul[@class='recipe-detail__list']").text.splitlines()
        except NoSuchElementException:
            recipe_ingredients = None
        try:
            recipe_directions = driver.find_element_by_xpath("//div[@class='recipe-detail__cms']").text.splitlines()
        except NoSuchElementException:
            recipe_directions = None
        recipe_image = driver.find_element_by_xpath("//img[@class='recipe-detail__img']").get_attribute('src')

        try:
            recipe_category_link = driver.find_element_by_xpath("//div[@class='recipe-detail__cms']//descendant::p/descendant::a").get_attribute('href')
        except NoSuchElementException:
            recipe_category_link = None
        try:
            recipe_category_name = driver.find_element_by_xpath("//div[@class='recipe-detail__cms']//descendant::p/descendant::a").text.replace(' recipes','')
        except NoSuchElementException:
            recipe_category_name = None
        
        try:
            recipe_serving = driver.find_element_by_xpath("//li[@class='recipe-detail__meta-item recipe-detail__meta-item_servings']").text
        except NoSuchElementException:
            recipe_serving = None
        try:
            recipe_duration = driver.find_element_by_xpath("//li[@class='recipe-detail__meta-item recipe-detail__meta-item_time']").text
        except NoSuchElementException:
            recipe_duration = None
        try:
            recipe_cal_per_serv = driver.find_element_by_xpath("//li[@class='recipe-detail__meta-item recipe-detail__meta-item_calories']").text
        except NoSuchElementException:
            recipe_cal_per_serv = None

        try:
            energy = driver.find_element_by_xpath("//span[@class='recipe-detail__nutrition-value recipe-detail__nutrition-value_small']").text.splitlines()[1]
        except NoSuchElementException:
            energy = None
        try:
            nutrition_facts_eles = driver.find_elements_by_xpath("//span[@class='recipe-detail__nutrition-value']")
            nutrition_facts = [nutrition_fact.text for nutrition_fact in nutrition_facts_eles]
        except NoSuchElementException:
            nutrition_facts = None
            
        nutrition_facts = [energy] + nutrition_facts
        
        try:
            nutrition_facts_perc_eles = driver.find_elements_by_xpath("//span[@class='recipe-detail__nutrition-percentage-item']")
            nutrition_facts_perc = [nutrition_fact_perc.text for nutrition_fact_perc in nutrition_facts_perc_eles]
        except NoSuchElementException:
            nutrition_facts = None
        
        try:
            addl_nutrition_facts_eles = driver.find_elements_by_xpath("//strong[@class='recipe-detail__nutrition-text_bold']")
            addl_nutrition_facts = [addl_nutrition_fact.text for addl_nutrition_fact in addl_nutrition_facts_eles]
        except NoSuchElementException:
            addl_nutrition_facts = None

        nutrition_facts_names = ['energy', 'fat', 'saturates', 'sugar', 'salt']
        addl_nutrition_facts_names = ['carbohydrate', 'protein', 'fibre']

        nutrition_facts_dict = dict(zip(nutrition_facts_names, nutrition_facts))
        nutrition_facts_perc_dict = dict(zip(nutrition_facts_names, nutrition_facts_perc))
        addl_nutrition_facts_dict = dict(zip(addl_nutrition_facts_names, addl_nutrition_facts))


        var = [index, recipe_names[counter], recipe_links[counter], recipe_types[counter], recipe_description, 
               recipe_ingredients, recipe_directions, recipe_image, recipe_category_name, recipe_serving, recipe_duration,
               recipe_cal_per_serv, nutrition_facts_dict, nutrition_facts_perc_dict, addl_nutrition_facts_dict]

        header = ['tesco_id', 'recipe_name', 'recipe_url', 'recipe_type', 'recipe_description', 'recipe_ingredients',
                  'recipe_directions', 'recipe_image', 'recipe_category', 'recipe_serving_count', 'recipe_duration', 
                  'recipe_cal_per_serv', 'recipe_nutrition_facts','recipe_nutrition_facts_percent', 'recipe_addl_nutrition_facts']

        write_to_file(tesco_results, var, header)
        index += 1
        time.sleep(randint(5,8))
        
driver.close()

In [ ]:
addl_nutrition_facts

# Trader Joe's

In [ ]:
trader_url = 'https://www.traderjoes.com/recipes'
trader_joes_outfile = 'tjs_recipes.csv'

options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)

driver.get(trader_url)

categories_ele = driver.find_elements_by_xpath("//div[@class='row text-center']//descendant::div/descendant::a")
categories = {category.text : category.get_attribute('href') for category in categories_ele}

category_name = list(categories.keys())
category_link = list(categories.values())

for category_counter in range(0, len(categories)):
    all_recipes = {}
    driver.get(category_link[category_counter])
    total_recipes = driver.find_element_by_xpath("//div[@class='article pagination-container']//descendant::span").text

    recipe_num_regex = re.compile(r'\: (\d*)')
    num_of_recipes = int(recipe_num_regex.search(total_recipes).group(1))
    num_of_pages = math.ceil(num_of_recipes/6)

    what_page = 1
    while what_page <= num_of_pages:
        recipes_ele = driver.find_elements_by_xpath("//div[@class='row text-center']//descendant::div/descendant::a")
        recipes = {recipe.text:recipe.get_attribute('href') for recipe in recipes_ele}
        all_recipes.update(recipes)

        try:
            next_page = driver.find_element_by_xpath("//a[@rel='next']").get_attribute('href')
        except NoSuchElementException:
            next_page = None

        what_page +=1

        if next_page:
            driver.get(next_page)
        else:
            pass
        
    recipe_names = list(all_recipes.keys())
    recipe_links = list(all_recipes.values())
    
        
    for recipe_counter in range(0, len(all_recipes)):
        driver.get(recipe_links[recipe_counter])
        try:
            recipe_image = driver.find_element_by_xpath("//div[@class='image-holder border-image image-border-detected']//descendant::img").get_attribute('src')
        except NoSuchElementException:
            recipe_image = None
        try:
            recipe_description = driver.find_element_by_xpath("//p[@class='space-top-thin']//following-sibling::p/descendant::em").text
        except NoSuchElementException:
            recipe_description = None

        try:
            recipe_overview = driver.find_element_by_xpath("//*[contains(text(), 'Serves: ')]//parent::p").text.splitlines()[0]
        except NoSuchElementException:
            recipe_overview = None
        try:
            recipe_ingredients = driver.find_element_by_xpath("//ul[@class='bullet-list']").text.splitlines()
        except NoSuchElementException:
            recipe_ingredients = None
        try:
            recipe_directions = driver.find_element_by_xpath("//div[@class='article ']").text.splitlines()
        except NoSuchElementException:
            recipe_directions = None
        try:
            recipe_tags = driver.find_elements_by_xpath("//a[@class='reltag']")
            recipe_tags = [recipe_tag.text for recipe_tag in recipe_tags]
        except NoSuchElementException:
            recipe_tags = None

        var = [index, recipe_category[category_counter], recipe_names[recipe_counter], recipe_links[recipe_counter], recipe_image, recipe_description, recipe_overview, recipe_ingredients, recipe_directions, recipe_tags]
        header = ['tjs_id', 'recipe_category', 'recipe_name', 'recipe_link', 'recipe_image', 'recipe_description', 'recipe_overview', 'recipe_ingredients', 'recipe_directions', 'recipe_tags']

        write_to_file(trader_joes_outfile, var, header)
        index += 1

# PCC Community Markets

In [ ]:
pcc_outfile = 'pcc_recipes.csv'

pcc_url = 'https://www.pccmarkets.com/?s&search_type=recipe&filter_dish_type=377'
main_page = 'https://www.pccmarkets.com'

options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)
print('Browser opened...')
driver.get(pcc_url)

current_page = 1
last_page = int(driver.find_element_by_xpath("//li[@class='page-item disabled']//following-sibling::li").text)

all_recipes = {}
while current_page <= last_page:
    recipe_ele = driver.find_elements_by_xpath("//div[@class='card-block pcc-card-block']//descendant::h2/descendant::a")
    recipe_name_n_link = {recipe.text:recipe.get_attribute('href') for recipe in recipe_ele}
    all_recipes.update(recipe_name_n_link)
    current_page += 1
    
    try:
        next_page = driver.find_element_by_xpath("//a[@class='next page-link']").get_attribute('href')
    except NoSuchElementException:
        next_page = None
        
    if next_page:
        driver.get(next_page)

print('Retrieved all recipes...')
        
recipe_names = list(all_recipes.keys())
recipe_links = list(all_recipes.values())

index = 100000
for counter in range(0, len(all_recipes)):

    driver.get(recipe_links[counter])

    try:
        author = driver.find_element_by_xpath("//header[@class='mb-md hidden-sm-down']//descendant::p").text
    except NoSuchElementException:
        author = None
    try:
        recipe_image = driver.find_element_by_xpath("//div[@class='pcc-content-bleed embed-responsive embed-responsive-3by2']//descendant::picture/descendant::img").get_attribute('src')
    except NoSuchElementException:
        recipe_image = None

    try:
        recipe_overview = driver.find_element_by_xpath("//ul[@class='pcc-list-md mb-md list-unstyled']").text
    except NoSuchElementException:
        recipe_overview = None

    try:
        recipe_description = driver.find_element_by_xpath("//p[@itemprop='description']").text
    except NoSuchElementException:
        recipe_description = None
    try:
        recipe_tags = driver.find_element_by_xpath("//ul[@class='list-unstyled pcc-recipe-tags']").text
    except NoSuchElementException:
        recipe_tags = None

    try:
        recipe_ingredients = driver.find_elements_by_xpath("//li[@itemprop='recipeIngredient']")
        recipe_ingredients = [recipe_ingredient.text for recipe_ingredient in recipe_ingredients]
    except NoSuchElementException:
        recipe_ingredients = None
    try:
        recipe_directions = driver.find_elements_by_xpath("//p[@itemprop='recipeInstructions']")
        recipe_directions = [recipe_direction.text for recipe_direction in recipe_directions]
    except:
        recipe_directions = None
        
    var = [index, recipe_names[counter].title(), recipe_links[counter], author, recipe_image, recipe_overview, recipe_description, recipe_tags, recipe_ingredients, recipe_directions, main_page]
    header = ['pcc_id', 'recipe_name', 'recipe_links', 'author', 'recipe_image', 'recipe_overview', 'recipe_description', 'recipe_tags', 'recipe_ingredients', 'recipe_directions', 'external_url']
    
    write_to_file(pcc_outfile, var, header)
    
    index += 1
    
print('Process Completed.')

# Natural Grocers

In [ ]:
url = 'https://www.naturalgrocers.com/recipe-finder?sort_by=nid&sort_order=DESC&search_api_fulltext=&page=0x'

# Binging With Babish
https://www.bingingwithbabish.com/recipes

In [26]:
b_with_b_main_page = 'https://www.bingingwithbabish.com/'
b_with_b_url = 'https://www.bingingwithbabish.com/recipes'

b_with_b_outfile = 'b_with_b_recipes.csv'

options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)

driver.get(b_with_b_url)
print('Browser opened...')

recipe_thumbnails = driver.find_elements_by_xpath("//div[@class='main-image-wrapper']//descendant::a/descendant::div/descendant::img")
recipe_thumbnails = [recipe_thumbnail.get_attribute('src') for recipe_thumbnail in recipe_thumbnails]

date_published = driver.find_elements_by_xpath("//time[@class='published']")
date_published = [date.get_attribute('datetime') for date in date_published]

recipe_eles = driver.find_elements_by_xpath("//div[@class='recipe-title-wrapper']//descendant::div/descendant::a")
recipe_name_n_link = {recipe.text:recipe.get_attribute('href') for recipe in recipe_eles}
print('Retrieved all recipes...')

recipe_names = list(recipe_name_n_link.keys())
recipe_links = list(recipe_name_n_link.values())

index = 100000
for counter in range(0, len(recipe_name_n_link)):
    driver.get(recipe_links[counter])
    recipe_parts = driver.find_elements_by_xpath("//div[@class='sqs-block html-block sqs-block-html']")[:3]
    parts = ['overview', 'ingredients', 'directions']
    recipe_dict = {parts[counter]:recipe_parts[counter].text for counter in range(0, len(recipe_parts))}
    sections = [section.splitlines() for section in list(recipe_dict.values())]

    var = [index, recipe_names[counter], recipe_links[counter], recipe_thumbnails[counter], date_published[counter], sections[0], sections[1], sections[2], b_with_b_main_page]
    header = ['b_with_b_id', 'recipe_name', 'recipe_link', 'recipe_thumbnail', 'date_published', 'recipe_description', 'recipe_ingredients', 'recipe_directions', 'external_url']

    write_to_file(b_with_b_outfile, var, header)
    
    index += 1

driver.close()

print('Process Completed.')

Browser opened...
Retrieved all recipes...
Process Completed.


# Once Upon a Chef by Jenn Segal
https://www.onceuponachef.com/recipes

# Five Heart Home
https://www.fivehearthome.com/recipes/

# Striped Spatula
https://stripedspatula.com/category/appetizers

# Dinner at the Zoo
https://www.dinneratthezoo.com/all-recipes/

# Kitchn (6216)
https://www.thekitchn.com/search?q=&filter=recipes

# Central Market
https://centralmarket.com/recipes/browse/

# Kevin is Cooking
https://keviniscooking.com/category/collections/

# The Cozy Apron
https://thecozyapron.com/recipes/

# Garlic and Zest
https://www.garlicandzest.com/recipes/

# How Sweet Eats
https://www.howsweeteats.com/2019/12/roasted-kabocha-squash-pomegranate-pepita-relish/

# Domestic Superhero
https://domesticsuperhero.com/recipe-index/

# The Candid Appetite
https://www.thecandidappetite.com/recipe-index/

# Love & Olive Oil
https://www.loveandoliveoil.com/recipe-index

# Lively Table
https://livelytable.com/recipes-3/

# Drizzle and Dip
https://drizzleanddip.com/category/recipes/

# Love and Lemons
https://www.loveandlemons.com/recipes/